# LangGraph Node Testing Lab

**Purpose**: Interactive testing environment for individual LangGraph nodes during development.

**What I'm Testing**:
- `fetch_news_node`: Fetches crypto news from API and creates NewsItem objects
- `check_cache_node`: Checks for cached news items
- Individual node functionality before integration

**Why Notebook**:
- Keep state between cells for iterative testing
- Visual feedback and debugging
- Quick experimentation without full test suite

**Author**: Peyman Kh  
**Date**: September 2025


In [1]:
# Add project root to Python path for module imports
# This allows importing from 'src' directory when running notebook from subdirectories
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
# Import libraries
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

from src.state import GraphState
from src.config.config import config
from src.nodes.fetch_news import fetch_news_node
from src.nodes.check_cache import check_cache_node
from src.nodes.sentiment_analysis import sentiment_analysis_node
from src.nodes.write_to_database import write_to_database_node

2025-09-20 20:10:27 | INFO | src.config.logging_config | Logging initialized successfully.
2025-09-20 20:10:27 | INFO | root | Configuration loaded for DEVELOPMENT environment.


In [3]:
os.environ["OPENAI_API_KEY"] = config.model_api_key.get_secret_value()

# 1. Create Graph

In [4]:
# Create Graph
builder = StateGraph(GraphState)

builder.add_node("fetch_news", fetch_news_node)
builder.add_node("check_cache", check_cache_node)
builder.add_node("sentiment_analysis", sentiment_analysis_node)
builder.add_node("write_to_database", write_to_database_node)

builder.add_edge(START, "fetch_news")
builder.add_edge("fetch_news", "check_cache")
builder.add_edge("check_cache", "sentiment_analysis")
builder.add_edge("sentiment_analysis", "write_to_database")
builder.add_edge("write_to_database", END)

memory = InMemorySaver()
graph = builder.compile(checkpointer=memory)

In [6]:
initial_state = GraphState(
    raw_news=[], 
    cache=[], 
    cache_hit=0, 
    unseen_news=[], 
    processed_news=[], 
    database_write_success=False, 
    telegram_notification_success=False
)
thread = {"configurable": {"thread_id": "test"}}

---
## Test Node 1: Fetch News Node

In [7]:
# invoke first node
result1 = graph.nodes["fetch_news"].invoke(initial_state, thread)

2025-09-20 20:11:25 | INFO | src.nodes.fetch_news | Fetching latest cryptocurrency news...
2025-09-20 20:11:26 | INFO | src.nodes.fetch_news | Successfully fetched latest cryptocurrency news.


In [8]:
result1

{'raw_news': [NewsItem(id='381e533c-11cb-57df-860d-1b8251d5e556', title='2 Examples Of How Better Blockchains Are Key For Wider Crypto Adoption', text='Legislating bitcoin purchases by the U.S. government might be exciting, but technical blockchain improvements remain essential for wider adoption', source_name='Forbes', news_url='https://www.forbes.com/sites/digital-assets/2025/09/20/2-examples-of-how-better-blockchains-are-key-for-wider-crypto-adoption/', image_url='https://crypto.snapi.dev/images/v1/r/l/9/2-examples-of-how-better-block-780224.jpg', timestamp=datetime.datetime(2025, 9, 20, 11, 52, 44, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000)))),
  NewsItem(id='47f890bf-2b91-5068-bf03-6b77d457637f', title='Canada Seizes $40M In Crypto From TradeOgre After Europol Tip', text="Canadian authorities have seized a record $40 million in crypto from cryptocurrency exchange TradeOgre. The seizure is the country's largest crypto seizure to date, and comes after a year

---
## Test Node 2: Check Cache

In [9]:
result2 = graph.nodes["check_cache"].invoke(result1, thread)

2025-09-20 20:11:28 | INFO | root | Initializing database connection...
2025-09-20 20:11:28 | INFO | root | Database connection initialized successfully.
2025-09-20 20:11:28 | INFO | src.nodes.check_cache | Cache loaded successfully with 10 items.


In [10]:
result2

{'cache': ['efd120f1-d744-5103-9250-d4a7940a38fa',
  'ce1cc8ef-ad31-594b-9ca0-db80da948661',
  'f25ffaca-b3c2-55f2-9500-88c756fdf409',
  'af9ab8cc-e58d-5760-8c7a-96b016076037',
  'a4c92b0c-d146-557b-b2c3-ef0712c1088f',
  'b205d519-1475-5741-9c28-5d9992e6fa6f',
  'f2ae039d-f959-5eef-ae58-d61b06621096',
  '16dbd9a6-bdb7-5f12-8a0b-08797dd94153',
  '2687f988-0bf4-5931-a92b-e29eab4df0e8',
  '74a08349-6034-5993-a633-9377a5cfed7b'],
 'cache_hit': 6,
 'unseen_news': [NewsItem(id='381e533c-11cb-57df-860d-1b8251d5e556', title='2 Examples Of How Better Blockchains Are Key For Wider Crypto Adoption', text='Legislating bitcoin purchases by the U.S. government might be exciting, but technical blockchain improvements remain essential for wider adoption', source_name='Forbes', news_url='https://www.forbes.com/sites/digital-assets/2025/09/20/2-examples-of-how-better-blockchains-are-key-for-wider-crypto-adoption/', image_url='https://crypto.snapi.dev/images/v1/r/l/9/2-examples-of-how-better-block-780224

In [11]:
len(result2["unseen_news"])

4

---
## Test Node 3: Sentiment Analysis

In [12]:
result3 = graph.nodes["sentiment_analysis"].invoke(result2, thread)

2025-09-20 20:11:31 | INFO | src.nodes.sentiment_analysis | Processing 4 news items...
2025-09-20 20:11:33 | INFO | httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-20 20:11:33 | INFO | src.nodes.sentiment_analysis | Successfully processed news item: 381e533c-11cb-57df-860d-1b8251d5e556
2025-09-20 20:11:34 | INFO | httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-20 20:11:34 | INFO | src.nodes.sentiment_analysis | Successfully processed news item: 47f890bf-2b91-5068-bf03-6b77d457637f
2025-09-20 20:11:35 | INFO | httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-20 20:11:35 | INFO | src.nodes.sentiment_analysis | Successfully processed news item: 2930090f-a511-5603-90b6-9075d441de19
2025-09-20 20:11:37 | INFO | httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-20 20:11:37 | INFO | src.nodes.sentiment_ana

In [13]:
result3

{'processed_news': [ProcessedNewsItem(id='381e533c-11cb-57df-860d-1b8251d5e556', title='2 Examples Of How Better Blockchains Are Key For Wider Crypto Adoption', text='Legislating bitcoin purchases by the U.S. government might be exciting, but technical blockchain improvements remain essential for wider adoption', source_name='Forbes', news_url='https://www.forbes.com/sites/digital-assets/2025/09/20/2-examples-of-how-better-blockchains-are-key-for-wider-crypto-adoption/', image_url='https://crypto.snapi.dev/images/v1/r/l/9/2-examples-of-how-better-block-780224.jpg', sentiment=<Sentiment.NEUTRAL: 'NEUTRAL'>, importance=<Importance.MEDIUM: 'MEDIUM'>, is_market_relevant=True, timestamp=datetime.datetime(2025, 9, 20, 11, 52, 44, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000)))),
  ProcessedNewsItem(id='47f890bf-2b91-5068-bf03-6b77d457637f', title='Canada Seizes $40M In Crypto From TradeOgre After Europol Tip', text="Canadian authorities have seized a record $40 million 

---
## Test Node 4: Write to Database

In [14]:
result4 = graph.nodes["write_to_database"].invoke(result3, thread)

2025-09-20 20:11:45 | INFO | root | Successfully inserted 4 news.
2025-09-20 20:11:45 | INFO | src.nodes.write_to_database | Successfully added 4 news items to the database.


In [15]:
result4

{'database_write_success': True}